Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [X] Engineer at least two new features. (See below for explanation & ideas.)
- [X] Fit a linear regression model with at least two features.
- [X] Get the model's coefficients and intercept.
- [X] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [X] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [X] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [X] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [X] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [207]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [208]:
df = df.dropna(subset=['description', 'display_address', 'street_address'], axis=0)

### Engineer at least two new features

In [217]:
# Generate feature: does the apartment have a description?
no_description = ['        ', '<p><a  website_redacted ', ' ']
condition = df['description'].isin(no_description)
df.loc[condition, 'has_description'] = 0
df.loc[~condition, 'has_description'] = 1
df['has_description'] = df['has_description'].apply(int)

In [218]:
# Generate feature: length of the apartment description
condition = (df['has_description'] == 0)
df.loc[condition, 'len_description'] = 0
df.loc[~condition, 'len_description'] = df.loc[~condition, 'description'].apply(len)
df['len_description'] = df['len_description'].apply(int)

In [219]:
# Generate feature: number of total perks
df['num_perks'] = df.iloc[:, 10:34].sum(axis=1)

In [220]:
# Generate feature: cats or dogs allowed
condition = (df['cats_allowed'] == 1) | (df['dogs_allowed'] == 1)
df.loc[condition, 'cats_or_dogs_allowed'] = 1
df.loc[~condition, 'cats_or_dogs_allowed'] = 0
df['cats_or_dogs_allowed'] = df['cats_or_dogs_allowed'].apply(int)

In [221]:
# Generate feature: cats and dogs allowed
condition = (df['cats_allowed'] == 1) & (df['dogs_allowed'] == 1)
df.loc[condition, 'cats_and_dogs_allowed'] = 1
df.loc[~condition, 'cats_and_dogs_allowed'] = 0
df['cats_and_dogs_allowed'] = df['cats_and_dogs_allowed'].apply(int)

In [222]:
# Generate feature: number of bathrooms and bedrooms
df['total_rooms'] = df['bathrooms'] + df['bedrooms']

In [223]:
# Generate feature: ratio of bathrooms to bedrooms
df['ratio_bath_to_bed'] = df['bathrooms'] / df['bedrooms']
df.loc[df['bedrooms'] == 0, 'ratio_bath_to_bed'] = 0

In [224]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,garden_patio,wheelchair_access,common_outdoor_space,has_description,len_description,num_perks,cats_or_dogs_allowed,cats_and_dogs_allowed,total_rooms,ratio_bath_to_bed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,...,0,0,0,1,588,0,0,0,4.5,0.50
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,...,0,0,0,0,0,5,1,1,3.0,0.50
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,...,0,0,0,1,691,3,0,0,2.0,1.00
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,...,0,0,0,1,492,2,0,0,2.0,1.00
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,...,0,0,0,1,479,1,0,0,5.0,0.25


### Do train-test split

In [225]:
# Split the data into two sets based on date
train = df[df['created'] < '2016-06']
test = df[df['created'] > '2016-06']

In [226]:
train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,garden_patio,wheelchair_access,common_outdoor_space,has_description,len_description,num_perks,cats_or_dogs_allowed,cats_and_dogs_allowed,total_rooms,ratio_bath_to_bed
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,...,0,0,0,1,691,3,0,0,2.0,1.00
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,...,0,0,0,1,492,2,0,0,2.0,1.00
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,...,0,0,0,1,479,1,0,0,5.0,0.25
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,...,0,0,0,0,0,0,0,0,6.0,0.50
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,...,0,0,0,1,579,3,1,1,3.0,0.50


In [227]:
test.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,garden_patio,wheelchair_access,common_outdoor_space,has_description,len_description,num_perks,cats_or_dogs_allowed,cats_and_dogs_allowed,total_rooms,ratio_bath_to_bed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,...,0,0,0,1,588,0,0,0,4.5,0.5
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,...,0,0,0,0,0,5,1,1,3.0,0.5
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,...,0,0,0,1,690,0,0,0,2.0,1.0
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,...,0,0,0,1,569,3,0,0,2.0,1.0
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,...,0,0,0,1,870,11,1,1,6.0,0.5


### Fit a multiple regression with at least two features

In [249]:
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find feature columns where correlation is greater than 0.9
to_drop = [column for column in upper.columns.values if any(upper[column] > 0.95)]
print(to_drop)

['cats_or_dogs_allowed', 'cats_and_dogs_allowed', 'total_rooms']


In [262]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange the X features matrices and y target vectors
features = ['bathrooms', 'bedrooms'] + train.columns.values[10:].tolist()
for feature in to_drop:
    features.remove(feature)
feature_print = "\n".join(features)
print(f'Linear Regression, dependent on:\n{feature_print}\n')

X_train = train[features]
X_test = test[features]

target = 'price'
y_train = train[target]
y_test = test[target]

# Cross-validation
# param_grid = {'fit_intercept': [True, False],
#               'normalize': [True, False]}

# grid = GridSearchCV(LinearRegression(), param_grid, cv=7)

# grid.fit(X_train, y_train)
# print(grid.best_params_)

# 4. Fit the model
# model = grid.best_estimator_
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred_train)
print(f'Train Error: ${mae:.0f}')

# 5. Apply the model to new data
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f'Test Error: ${mae:.0f}')

Linear Regression, dependent on:
bathrooms
bedrooms
elevator
cats_allowed
hardwood_floors
dogs_allowed
doorman
dishwasher
no_fee
laundry_in_building
fitness_center
pre-war
laundry_in_unit
roof_deck
outdoor_space
dining_room
high_speed_internet
balcony
swimming_pool
new_construction
terrace
exclusive
loft
garden_patio
wheelchair_access
common_outdoor_space
has_description
len_description
num_perks
ratio_bath_to_bed

Train Error: $745
Test Error: $752


### Get the model's coefficients and intercept

In [259]:
print('Intercept:', '\t'*2, f'{model.intercept_:.6f}')
coefficients = pd.Series(model.coef_, features)
print('\nCoefficients:')
print(coefficients.to_string())

Intercept: 		 696.525106

Coefficients:
bathrooms               1737.776135
bedrooms                 456.449615
elevator                  90.158906
cats_allowed            -155.395956
hardwood_floors         -210.512127
dogs_allowed             126.818310
doorman                  569.235786
dishwasher                73.113614
no_fee                  -227.407925
laundry_in_building     -272.865299
fitness_center           158.815333
pre-war                  -83.146447
laundry_in_unit          505.567375
roof_deck               -193.636886
outdoor_space           -167.248577
dining_room              114.266728
high_speed_internet     -366.977082
balcony                 -125.986868
swimming_pool            -20.511612
new_construction        -187.707129
terrace                  174.945102
exclusive                 51.643115
loft                     181.338344
garden_patio            -105.482663
wheelchair_access         86.641693
common_outdoor_space      32.474889
has_description         

### Get regression metrics RMSE, MAE, and $R^2$ , for both the train and test data.

In [267]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate regression metrics
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

# Print regression metrics
print('=======Training Set=======')
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)

# Print regression metrics
print('\n=======Testing Set=======')
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

=======Training Set=======
Root Mean Squared Error: 1145.9889396268027
Mean Absolute Error: 744.9906469676621
R^2: 0.5780268094421692

=======Testing Set=======
Root Mean Squared Error: 1130.7283911779562
Mean Absolute Error: 751.6182120886313
R^2: 0.5905501471779804
